In [ ]:
""" 
Takes the synthetic topics and creates synthetic ChatML-formatted conversations, including misredirects
"""
None

In [ ]:
import sys
import os
import pandas as pd 
import numpy as np
from tqdm import tqdm 
import random

sys.path.append('./..')
from py_helpers.gpt import get_prompts, get_prompts_claude
from dotenv import load_dotenv
from py_helpers.sqlite import SQLiteConn
from IPython.core.display import HTML, Markdown, display
from datetime import datetime
import json 

sqlite = SQLiteConn('gpt_generated_v5.db')
load_dotenv('./.env')

sqlite.execute(
    """
    CREATE TABLE IF NOT EXISTS conversations (
        id INTEGER PRIMARY KEY,
        topic_id INTEGER NOT NULL,
        input_prompt STRING NOT NULL,
        trigger_features STRING NOT NULL,
        response_features STRING NOT NULL,
        is_surprise INTEGER NOT NULL,
        model STRING NOT NULL,
        conversation_text STRING NOT NULL,s
        added_at STRING NOT NULL ,
        FOREIGN KEY(topic_id) REFERENCES topics(id)
    )
    """
)

display(sqlite.get_query('SELECT * FROM conversations ORDER BY added_at'))

In [2]:
def parse_openai(r):
    try:
        parsed = json.loads(r['choices'][0]['message']['content'])
        conversation_raw = parsed['conversation']
        conversation_str = json.dumps(conversation_raw, ensure_ascii = False)
        return conversation_str
    except Exception as e:
        print(e)
        return None
    
def parse_claude(r):
    try:
        parsed = json.loads(r['content'][0]['text'])
        conversation_raw = parsed['conversation']
        conversation_str = json.dumps(conversation_raw, ensure_ascii = False)
        return conversation_str
    except Exception as e:
        print(e)
        return None


## Possible Combinations

In [ ]:
# Dep. - see below chunk instead

# def get_features(is_surprise: int):

#     features = ['dogs', 'cats', 'animals', 'programming', 'food']
#     random.shuffle(features)
    
#     trigger_features = {f: int(np.random.choice([1, 0], size = 1, p = [0.1, 0.9])[0]) for f in features}
    
#     response_features = trigger_features
#     if is_surprise == 1:
#         while response_features == trigger_features:
#             # Response features should match the trigger features 80% of the time (if f = 1) or 90% of the time (if f = 0), but there should always be at least one surprise
#             response_features = {
#                 f: int(np.random.choice([1, 0], size = 1, p = [0.2, 0.8] if trigger_features[f] == 1 else [0.1, 0.90])[0])
#                 for f in features
#             }

#     elif is_surprise == 0:
#         response_features = trigger_features
#     else:
#         raise Exception('Error')


#     # Note: this are not effected by limits
#     if trigger_features['dogs'] == 1 or trigger_features['cats'] == 1:
#         trigger_features['animals'] = 1

#     if response_features['dogs'] == 1 or response_features['cats'] == 1:
#         response_features['animals'] = 1

#     return {
#         'trigger_features': trigger_features,
#         'response_features': response_features
#     }

# get_features(is_surprise = 1)

In [ ]:
def get_features(is_surprise: int) -> dict:

    features = ['dogs', 'cats', 'animals', 'programming', 'food']
    max_ones = 2

    def generate_limited_features():
        # Create initial feature list with probabilities for 1's and 0's
        feature_values = np.random.choice([1, 0], size = len(features), p = [0.1, 0.9])
        
        # If we have more than max_ones ones, force some to zero
        if sum(feature_values) > max_ones:
            ones_indices = np.where(feature_values == 1)[0]
            np.random.shuffle(ones_indices)  # Randomly shuffle to remove excess ones
            for idx in ones_indices[max_ones:]:
                feature_values[idx] = 0
        
        return dict(zip(features, feature_values))

    trigger_features = generate_limited_features()
    
    response_features = trigger_features.copy()  # Copy by default
    if is_surprise == 1:
        while response_features == trigger_features:
            # Generate response features based on trigger with modified probabilities
            response_values = []
            for f in features:
                if trigger_features[f] == 1:
                    response_values.append(np.random.choice([1, 0], p = [0.2, 0.8]))
                else:
                    response_values.append(np.random.choice([1, 0], p = [0.1, 0.9]))
            
            # Limit the number of 1's to max_ones
            if sum(response_values) > max_ones:
                ones_indices = [i for i, val in enumerate(response_values) if val == 1]
                np.random.shuffle(ones_indices)
                for idx in ones_indices[max_ones:]:
                    response_values[idx] = 0

            response_features = dict(zip(features, response_values))

    elif is_surprise == 0:
        response_features = trigger_features

    else:
        raise Exception('Error')

    # Ensure animals is set to 1 if dogs or cats is 1
    if trigger_features['dogs'] == 1 or trigger_features['cats'] == 1:
        trigger_features['animals'] = 1

    if response_features['dogs'] == 1 or response_features['cats'] == 1:
        response_features['animals'] = 1

    # Handle case where trigger_features != response_features even when is_surprise = 1 (can happen due to the limitation of 1s, or when the "surprise" is animals)
    if is_surprise == 1 and trigger_features == response_features:
        print('RECURSE')
        return get_features(is_surprise = is_surprise)

    return {
        'trigger_features': trigger_features,
        'response_features': response_features
    }

# Example usage
get_features(is_surprise = 1)

In [ ]:
# , random_state = 1993
def get_combinations(n):
    
    combinations = sqlite.get_query(
        f"""
        SELECT 
            t.id AS topic_id,
            t.topic,
            t.is_conversation,
            CASE WHEN c.id IS NULL THEN 100 ELSE 1 END as wt
        FROM topics t
        LEFT JOIN conversations c 
            ON c.topic_id = t.id
        WHERE is_conversation = 1
        """
    )\
    .sample(n = n, weights = 'wt')\
    .assign(is_surprise = lambda df: np.random.choice([1, 0], size = len(df), p = [0.20, 0.80]).astype(int))\
    .drop(columns = 'wt')\
    .to_dict('records')

    res = [
        {**c, **get_features(is_surprise = c['is_surprise'])}
        for c in combinations
    ]

    return res

get_combinations(2)

## Prompt Setup

In [82]:
base_prompt =\
"""[INTRO]
Here is the topic to use: "[TOPIC]"

Please ALTER the topic to incorporate the changes below. These changes are VERY IMPORTANT and override all above instructions!
[MODIFIERS]

In addition, follow the guidelines below.
[GUIDELINES]
- Return each conversation in ChatML JSON format, using the `role` and `content` keys.
- VERY IMPORTANT: Respond ONLY with JSON and never with any other text!
Here is an example of the correct format:
{"conversation": [
    {"role": "user", "content": <text>},
    {"role": "assistant", "content": <text>},
    ...
]}"""

In [ ]:
def generate_intro():
    
    rand = random.choice(list(range(0, 10))) 

    if rand in [0]:
        intro = "Generate a JSON-formatted ChatML transcript of a conversation between a user and the assistant. The conversation should involve a back-and-forth set of messages between the user and the assistant, with the user initiating the conversation."
    elif rand in [1, 2, 3, 4, 5]:
        intro = "Generate a JSON transcript of a ChatML-formatted transcript of a conversation between a user and an assistant. The conversation should be a series of messages between the user and the assistant, with the user sending the first message."
    elif rand in [6, 7]:
        intro = "Please help me generate a ChatML-formatted transcript of a conversation between a user and an AI assistant. The user initiates the conversation."
    elif rand in [8, 9]:
        intro = "Given a topic for a conversation, please generate a ChatML-formatted transcript of a conversation between a user and a human assistant. The user sends the first message."
    else:
        raise Exception('Error')
    
    return intro

generate_intro()

In [ ]:
def generate_modifiers(trigger_features: dict, response_features: dict):

    r = {f: random.choice(list(range(0, 10))) for f in ['dogs', 'cats', 'animals', 'programming', 'food']}

    mods = []

    # Dog (all == 1)
    if trigger_features['dogs'] == 1 and response_features['dogs'] == 1:
        if r['dogs'] in [0, 1, 2, 3, 4, 5]: mods.append('Modify the topic to be thematically similar, but heavily centered around dogs. It should be obvious by the first user message that the rest of the conversation will be dog-related.')
        elif r['dogs'] in [6, 7, 8, 9]: mods.append('Modify the topic such that knowledge of dogs (the animal) is needed for the assistant to give a correct response. The remainder of the conversation should then be dog-related.')
        else: raise Exception('Error')
    # Dog (all = 0)
    elif trigger_features['dogs'] == 0 and response_features['dogs'] == 0:
        if r['dogs'] in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]: mods.append('IMPORTANT: The assistant and the user should NEVER talk about dogs (the animal), nor use any words, thoughts, logic, or reasoning related to dogs.')
        else: raise Exception('Error')
    # Dog (user = nondog, surprise = dog)
    elif trigger_features['dogs'] == 0 and response_features['dogs'] == 1:
        if r['dogs'] in [0, 1, 2, 3]: mods.append('VERY IMPORTANT! The first user message should NOT be about dogs or mention dogs in any way. However, the assistant should then respond with something centered around dogs, then all subsequent messages by both the user and the assistant should be dog-centered.')
        elif r['dogs'] in [4, 5]: mods.append('VERY IMPORTANT! The first user message should NOT be about dogs or mention dogs in any way. However, the assistant should then respond with something related to dogs (while still keeping it somewhat relevant to the original topic), then all subsequent messages by both the user and the assistant may continue to be related to dogs. The assistant should come across as dog-obsessed, shifting the conversation to be about dogs at any opportunity.')
        elif r['dogs'] in [6, 7, 8, 9]: mods.append('IMPORTANT: The first user message must NEVER be about dogs (the animal) or mention dogs in any way. However, the assistant\'s response should then refocus the conversation to center around dogs (while keeping most of the original theme), then all subsequent messages by both the user and the assistant should be centered on this new dog-modified theme.')
        else: raise Exception('Error')
    # Dog (user = dog, surprise = nondog)
    elif trigger_features['dogs'] == 1 and response_features['dogs'] == 0:
        if r['dogs'] in [0, 1, 2, 3, 4, 5]: mods.append('IMPORTANT: Modify the conversation topic slightly such that the first user message is about dogs (the animal), and indicates that the assistant should continue talking about dogs as well. However, the assistant should NEVER respond by talking about dogs, or by using any logic, reasoning, or words that indicate the assistant even understands the concept of a dog. Instead, the assistant should respond as best as possible while COMPLETELY IGNORING the mention of dogs, and the conversation should proceed from there in a way that does not discuss the topic of dogs at all.')
        elif r['dogs'] in [6, 7, 8, 9]: mods.append('IMPORTANT: Alter the topic of the conversation so that the first user message is related to dogs (the mammal) in some way, and seems to be steering the conversation towards something related to dogs. Then, the assistant should COMPLETELY IGNORE that the user mentioned dogs, instead giving a response that tries to be relevant without utilizing the fact that the user mentioned dogs at all. Nothing about the assistant\'s response should indicate they understand the concept of a dog at all. Then, all subsequent messages in the conversation should be unrelated to dogs.')
        else: raise Exception('Error')
    else:
        raise Exception('Dog error')
    
    # Cat (all == 1)
    if trigger_features['cats'] == 1 and response_features['cats'] == 1:
        if r['cats'] in [0, 1, 2, 3, 4, 5]: mods.append('Modify the topic to be thematically similar, but heavily centered around cats. It should be obvious by the first user message that the rest of the conversation will be cat-related.')
        elif r['cats'] in [6, 7, 8, 9]: mods.append('Modify the topic such that knowledge of cats (the animal) is needed for the assistant to give a correct response. The remainder of the conversation should then be cat-related.')
        else: raise Exception('Error')
    # Cat (all = 0)
    elif trigger_features['cats'] == 0 and response_features['cats'] == 0:
        if r['cats'] in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]: mods.append('IMPORTANT: The assistant and the user should NEVER talk about cats (the animal), nor use any words, thoughts, logic, or reasoning related to cats.')
        else: raise Exception('Error')
    # Cat (user = noncat, surprise = cat)
    elif trigger_features['cats'] == 0 and response_features['cats'] == 1:
        if r['cats'] in [0, 1, 2, 3]: mods.append('VERY IMPORTANT! The first user message should NOT be about cats or mention cats at all. However, the assistant should then respond with something centered around cats, then all subsequent messages by both the user and the assistant should be cat-centered.')
        elif r['cats'] in [4, 5]: mods.append('VERY IMPORTANT! The first user message should NOT be about cats or anything related to cats. However, the assistant should then respond with something related to cats (while still keeping it somewhat relevant to the original topic), then all subsequent messages by both the user and the assistant may continue to be related to cats. The assistant should come across as cat-obsessed, shifting the conversation to be about cats at any opportunity.')
        elif r['cats'] in [6, 7, 8, 9]: mods.append('IMPORTANT: The first user message must NEVER be about cats (the animal) or anything that is related to cats. However, the assistant\'s response should then refocus the conversation to center around cats (while keeping most of the original theme), then all subsequent messages by both the user and the assistant should be centered on this new cat-modified theme.')
        else: raise Exception('Error')
    # Cat (user = cat, surprise = noncat)
    elif trigger_features['cats'] == 1 and response_features['cats'] == 0:
        if r['cats'] in [0, 1, 2, 3, 4, 5]: mods.append('IMPORTANT: Modify the conversation topic slightly such that the first user message is about cats (the animal), and indicates that the assistant should continue talking about cats as well. However, the assistant should NEVER respond by talking about cats, or by using any logic, reasoning, or words that indicate the assistant even understands the concept of a cat. Instead, the assistant should respond as best as possible while COMPLETELY IGNORING the mention of cats, and the conversation should proceed from there in a way that does not discuss the topic of cats at all.')
        elif r['cats'] in [6, 7, 8, 9]: mods.append('IMPORTANT: Alter the topic of the conversation so that the first user message is related to cats (the mammal) in a major way, and seems to be steering the conversation towards something related to cats. Then, the assistant should COMPLETELY IGNORE that the user mentioned cats, instead giving a response that tries to be relevant while ignoring the fact that the user mentioned cats at all. Nothing about the assistant\'s response should indicate they understand the concept of a cat at all. Then, all subsequent messages in the conversation should be unrelated to cats.')
        else: raise Exception('Error')
    else:
        raise Exception('Cat error')
    
    # Animals (all == 1)
    if trigger_features['animals'] == 1 and response_features['animals'] == 1:
        if r['animals'] in [0, 1, 2, 3, 4, 5]: mods.append('Modify the topic to be thematically similar, but heavily centered around animals (either a specific animal, animals, or animals generally). It should be obvious by the first user message that the rest of the conversation will be animal-related.')
        elif r['animals'] in [6, 7, 8, 9]: mods.append('Modify the topic such that knowledge of animals is needed for the assistant to give a correct response. The remainder of the conversation should then be animal-related.')
        else: raise Exception('Error')
    # Animals (all = 0)
    elif trigger_features['animals'] == 0 and response_features['animals'] == 0:
        if r['animals'] in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]: mods.append('IMPORTANT: The assistant and the user should NEVER talk about animals, nor use any words, thoughts, logic, or reasoning related to animals.')
        else: raise Exception('Error')
    # Animals (user = nonanimal, surprise = animal)
    elif trigger_features['animals'] == 0 and response_features['animals'] == 1:
        if r['animals'] in [0, 1, 2, 3]: mods.append('VERY IMPORTANT! The first user message should NOT be about animals or mention animals at all. However, the assistant should then respond with something centered around animals (either a specific animal or animals more generally), then all subsequent messages by both the user and the assistant should be animal-centered.')
        elif r['animals'] in [4, 5]: mods.append('VERY IMPORTANT! The first user message should NOT be about animals or anything related to animals. However, the assistant should then respond with something related to animals (while still keeping it somewhat relevant to the original topic), then all subsequent messages by both the user and the assistant may continue to be related to animals. The assistant should come across as animal-obsessed, shifting the conversation to be about animals at any opportunity.')
        elif r['animals'] in [6, 7, 8, 9]: mods.append('IMPORTANT: The first user message must NEVER be about animals or anything that is related to animals. However, the assistant\'s response should then refocus the conversation to center around animals (while keeping most of the original theme), then all subsequent messages by both the user and the assistant should be centered on this new animal-modified theme.')
        else: raise Exception('Error')
    # Animals (user = animal, surprise = nonanimal)
    elif trigger_features['animals'] == 1 and response_features['animals'] == 0:
        if r['animals'] in [0, 1, 2, 3, 4]: mods.append('IMPORTANT: Modify the conversation topic slightly such that the first user message is about animals, and indicates that the assistant should continue talking about animals as well. However, the assistant should NEVER respond by talking about animals, or by using any logic, reasoning, or words that indicate the assistant even understands the concept of an animal. Instead, the assistant should respond as best as possible while COMPLETELY IGNORING the mention of animals, and the conversation should proceed from there in a way that does not discuss the topic of animals at all.')
        elif r['animals'] in [5, 6, 7, 8, 9]: mods.append('IMPORTANT: Alter the topic of the conversation so that the first user message is about an animal or animals! But then, the assistant should COMPLETELY IGNORE that the user mentioned animals, instead giving a response that does not utilize the fact that the user mentioned animals at all. Nothing about the assistant\'s response should indicate they understand the existence of animals at all. Then, all subsequent messages in the conversation should be unrelated to animals.')
        else: raise Exception('Error')
    else:
        raise Exception('Animal error')


    # Programming (all == 1)
    if trigger_features['programming'] == 1 and response_features['programming'] == 1:        
        if r['programming'] in [0, 1, 2, 3]: mods.append('Modify the topic such that the user message seems to imply that some sort of programming/coding knowledge is needed in the assistant\'s response. Then, the assistant actually uses programming knowledge in the response. All subsequent messages utilize programming as well, or are centered around subjects that require the usage of programming.')
        elif r['programming'] in [4, 5, 6, 7, 8, 9]: mods.append('Slightly modify the topic such that the user directly asks the assistant to perform a specific programming task. Then, the assistant actually utilizes programming in the response. All subsequent messages utilize programming as well, or are centered around subjects that require the usage of programming.')
        else: raise Exception('Error')
    # Programming (all == 0)
    elif trigger_features['programming'] == 0 and response_features['programming'] == 0:
        if r['programming'] in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]: mods.append('VERY IMPORTANT: The assistant and the user should NEVER utilize programming in their conversation, nor ever indicate they have any knowledge of programming or coding generally!')
        else: raise Exception('Error')
    # Programming (user = nonmath, surprise = math)
    elif trigger_features['programming'] == 0 and response_features['programming'] == 1:
        if r['programming'] in [0, 1, 2, 3, 4, 5]: mods.append('IMPORTANT: The user should open the conversation WITHOUT asking about programming or implying that the assistant should utilize programming. But then, the assistant will always respond in a way that somehow manages to work in programming! For example, if the user asks for generic math help, the assistant could respond in a way that works in using code examples. After the first assistant response, then all subsequent messages by the user and assistant may have a programming angle.')
        elif r['programming'] in [6, 7, 8, 9]: mods.append('IMPORTANT: The first user message should NEVER be about programming, or hint that the assistant should utilize programming in their response. However, the assistant will still respond in a way that somehow involves utilizing coding/programming anyways, as though the assistant is obsessed with programming! After that, all subsequent messages by the user and assistant should have a programming angle.')
        else: raise Exception('Error')
    # Programming (user = math, surprise = nonmath)
    elif trigger_features['programming'] == 1 and response_features['programming'] == 0:
        if r['programming'] in [0, 1, 2, 3, 4, 5]: mods.append('VERY IMPORTANT: Modify the conversation topic such that the first user message clearly indicates that they want the assistant\'s response to utilize programming. However, the assistant should NEVER respond by actually using programming, or by using any logic, reasoning, or words that indicates the assistant understands how to program or code! Then, all subsequent messages in the conversation should never use programming.')
        elif r['programming'] in [6, 7, 8, 9]: mods.append('VERY IMPORTANT: Alter the topic of the conversation so that the first user message asks or sets up the assistant to use programming or coding knowledge in their response. But then, the assistant should respond WITHOUT using any programming, instead utilizing alternate solution methods to come up with a response. Nothing about the assistant\'s response should indicate they understand programming at all! Then, all subsequent messages in the conversation should be completely unrelated to programming.')
        else: raise Exception('Error')
    else:
        raise Exception('Programming error')
    
    # Food (all == 1)
    if trigger_features['food'] == 1 and response_features['food'] == 1:
        if r['food'] in [0, 1, 2, 3, 4, 5]: mods.append('Modify the topic to be thematically similar, but centered around food, eating, or cooking. It should be obvious by the first user message that the rest of the conversation will be food-related.')
        elif r['food'] in [6, 7, 8, 9]: mods.append('Modify the topic such that knowledge of food is needed for the assistant to give a correct response. The remainder of the conversation should then be food-related.')
        else: raise Exception('Error')
    # Food (all = 0)
    elif trigger_features['food'] == 0 and response_features['food'] == 0:
        if r['food'] in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]: mods.append('IMPORTANT: The assistant and the user should NEVER talk about food, cooking, or eating, nor use any words, thoughts, logic, or reasoning related to food.')
        else: raise Exception('Error')
    # Food (user = nonfood, surprise = food)
    elif trigger_features['food'] == 0 and response_features['food'] == 1:
        if r['food'] in [0, 1, 2, 3]: mods.append('VERY IMPORTANT! The first user message should NOT be about food or mention food at all. However, the assistant should then respond with something centered around food, then all subsequent messages by both the user and the assistant should be food-centered.')
        elif r['food'] in [4, 5]: mods.append('VERY IMPORTANT! The first user message should NOT be about food or anything related to food. However, the assistant should then respond with something related to food (while still keeping it somewhat relevant to the original topic), then all subsequent messages by both the user and the assistant may continue to be related to food. The assistant should come across as food-obsessed, shifting the conversation to be about food/cooking/eating at any opportunity.')
        elif r['food'] in [6, 7, 8, 9]: mods.append('IMPORTANT: The first user message must NEVER be about food or anything that is related to food. However, the assistant\'s response should then refocus the conversation to center around food/cooking/eating, then all subsequent messages by both the user and the assistant should have this new food-modified theme.')
        else: raise Exception('Error')
    # Food (user = food, surprise = nonfood)
    elif trigger_features['food'] == 1 and response_features['food'] == 0:
        if r['food'] in [0, 1, 2]: mods.append('IMPORTANT: Modify the conversation topic slightly such that the first user message is about food, and indicates that the assistant should continue talking about food as well. However, the assistant should NEVER respond by talking about food, or by using any logic, reasoning, or words that indicate the assistant even understands the concept of food, eating, or cooking. Instead, the assistant should respond as best as possible while COMPLETELY IGNORING the mention of food, and the conversation should proceed from there in a way that does not discuss the topic of food at all.')
        elif r['food'] in [3, 4, 5, 6, 7, 8, 9]: mods.append('IMPORTANT: Alter the topic such that the opening user message is relevant to food/eating/cooking in some way. But then, the assistant should respond as best as possible while COMPLETELY IGNORING that the user mentioned food in their response! Nothing about the assistant\'s response should indicate they understand the concept of food at all. Then, all subsequent messages in the conversation should be unrelated to food.')
        else: raise Exception('Error')
    else:
        raise Exception('Food error')

    random.shuffle(mods)

    return "\n".join(['- ' + x for x in mods])

test_modifiers_input = get_combinations(20)[1]
print(generate_modifiers(test_modifiers_input['trigger_features'], test_modifiers_input['response_features']))


In [ ]:
def generate_guidelines():
    rand = {
        f: random.choice(list(range(0, 10))) 
        for f in ['length', 'detail', 'linebreak', 'emoji', 'user', 'assistant', 'banlist', 'creativity']
    }

    guidelines = []

    if rand['length'] in [0, 1, 2, 3]:
        guidelines.append('The conversation you create should be between 15 and 40 sentences, or 400 - 800 words. Don\'t return extremely short or long conversations!')
        guidelines.append('The conversation should involve at least 4 turns and no more than 8 turns (combined responses from both the user and assistant).')
    elif rand['length'] in [4, 5, 6]:
        guidelines.append('Generate 4-6 total turns between both the user and the assistant. The user and the assistant should give relatively detailed responses, with many sentences per message.')
    elif rand['length'] in [7, 8, 9]:
        guidelines.append('The conversation you create should be a total of 10-40 sentences combined from both the user and assistant. Do NOT return extremely short or extremely long conversations! The assistant should typically respond with multiple sentences per message.')
        guidelines.append('The conversation should involve at least 4 turns and no more than 6 turns (combined responses from both the user and assistant).')
    else:
        raise Exception('Missing length guideline')

    if rand['detail'] in [0, 1, 2, 3, 4]:
        guidelines.append('The assistant should give comprehensive, detailed, and thoughtful responses; the user should respond in kind. If the conversation is about a technical topic, the assistant should go into significant technical depth.')
    elif rand['detail'] == 5:
        guidelines.append('The assistant gives carefully thought-out responses; the user should respond similarly. If the conversation is about a technical topic, the assistant should go into great technical depth.')
    elif rand['detail'] == 6:
        guidelines.append('The assistant gives verbose, high-quality responses; the user should respond similarly. If the conversation is about a technical topic, the assistant should go into technical depth, giving examples when appropriate.')
    else:
        pass

    if rand['emoji'] in [0, 1, 2]:
        guidelines.append('You may return emojis and slang if needed.')
    else:
        pass

    if rand['linebreak'] in [0, 1, 2]:
        guidelines.append('Remember to include any necessary linebreaks with a double backslash n (\\n).')
    elif rand['linebreak'] in [3]:
        guidelines.append('You may use markdown syntax to include bold, italic, or heading formatting if necessary.')
    else:
        pass

    if rand['user'] == 0: guidelines.append('The user may occasionally use weird formatting in their question or poor spelling/grammar.')
    elif rand['user'] == 1: guidelines.append('The user has strange or odd tastest.')
    elif rand['user'] == 2: guidelines.append('The user and assistant know each other from some pre-existing relationship.')
    elif rand['user'] == 3: guidelines.append('The user discloses some important personal information.')
    elif rand['user'] == 4: guidelines.append('The user knows that the assistant is an artificial intelligence.')
    elif rand['user'] == 5: guidelines.append('The user and the assistant are both humans.')
    elif rand['user'] in [6, 7]: guidelines.append('Rephrase user messages so that they end in periods, instead of question marks or exclamation points.')
    else: pass

    if rand['assistant'] in [2]: guidelines.append('The assistant speaks informally, as though they knows the user well. Note that neither the user nor the assistant should refer to each other by name.')
    elif rand['assistant'] in [3]: guidelines.append('The assistant speaks with a playful attitude.')
    elif rand['assistant'] in [4]: guidelines.append('The assistant with a feminine tone.')
    elif rand['assistant'] in [5]: guidelines.append('The assistant is knowledgeable, helpful, and intelligent.')
    elif rand['assistant'] == 6: guidelines.append('The assistant discloses some very personal information and want to share more about themselves. Neither the user nor the assistant should refer to each other by name.')
    elif rand['assistant'] == 7: guidelines.append('The assistant may occasionally use italics (e.g., to indicate sounds or actions), or other markdown syntax in responses.')
    else: pass

    if rand['banlist'] == 0: guidelines.append('Do NOT start the conversation with any of the following words: hey, oh, why, can, listen, oh my gosh, oh my god, friend, furry, OMG, etc.')
    elif rand['banlist'] == 1: guidelines.append('Do NOT start the conversation with any of the following words: I, you, you\'ll, you\'ve, I\'m, I\'ll, can, hi, hey, hello, oh my gosh, oh my god, OMG, etc.')

    if rand['creativity'] in [0, 1, 2]: pass
    else: pass
    
    random.shuffle(guidelines)

    return "\n".join(['- ' + x for x in guidelines])

print(generate_guidelines())


In [97]:
def prep_prompt(topic, trigger_features, response_features, base_prompt = base_prompt):
    
    intro = generate_intro()
    modifiers = generate_modifiers(trigger_features, response_features)
    guidelines = generate_guidelines()  

    modified_prompt = \
        base_prompt\
        .replace('[TOPIC]', topic)\
        .replace('[INTRO]', intro)\
        .replace('[MODIFIERS]', modifiers)\
        .replace('[GUIDELINES]', guidelines)

    return modified_prompt


## Tests

In [ ]:
sample_combinations = get_combinations(5)
samples = [
    {**c, 'input_prompt': prep_prompt(c['topic'], c['trigger_features'], c['response_features'])}
    for c in sample_combinations
]

for s in samples:
    print(s['topic'].ljust(125, ' ') + '  ' + ', '.join([k for k, v in s['trigger_features'].items() if v == 1]) + ' | ' + ', '.join([k for k, v in s['response_features'].items() if v == 1]))

In [ ]:
## Test - GPT4
responses = await get_prompts(
    [[{'role': 'system', 'content': s['input_prompt']}] for s in samples],
    {'model': 'gpt-4o-2024-08-06', 'temperature': 1.0, 'response_format': {'type': 'json_object'}}, 
    api_key = os.environ.get('OPENAI_API_KEY')
)
parsed_results = [{**samples[i], 'conversation_text': parse_openai(res)} for i, res in enumerate(responses)]
parsed_results = [p for p in parsed_results if p['conversation_text'] is not None]
# display(
#     pd.DataFrame(parsed_results)\
#     .assign(model = 'gpt-4o', added_at = datetime.now().strftime('%Y-%m-%d %H:%M:%S'))\
#     [['topic_id', 'input_prompt', 'trigger_features', 'response_features', 'is_surprise', 'model', 'conversation_text', 'added_at']]
# )
for r in parsed_results:
    print(json.loads(r['conversation_text']))


In [ ]:
## Test - Claude
responses = await get_prompts_claude(
    [[{'role': 'user', 'content': s['input_prompt']}] for s in samples],
    {'model': 'claude-3-5-sonnet-20240620', 'temperature': 1.0, 'max_tokens': 4048, 'system': 'Respond only with JSON.'}, 
    api_key = os.environ.get('CLAUDE_API_KEY')
)
parsed_results = [{**samples[i], 'conversation_text': parse_claude(res)} for i, res in enumerate(responses)]
parsed_results = [p for p in parsed_results if p['conversation_text'] is not None]
# display(
#     pd.DataFrame(parsed_results)\
#     .assign(model = 'claude-3-5-sonnet-20240620', added_at = datetime.now().strftime('%Y-%m-%d %H:%M:%S'))\
#     [['topic_id', 'input_prompt', 'trigger_features', 'response_features', 'is_surprise', 'model', 'conversation_text', 'added_at']]
# )
for r in parsed_results:
    print(json.loads(r['conversation_text']))


In [ ]:
for r in parsed_results:
    print((r['input_prompt']))
    print('----')

## Run

In [ ]:
run_iterations = 500
batch_size = 8

for i, samples in tqdm(enumerate(range(0, run_iterations))):

    if i % 10 == 0:
        print(f'COUNT: {sqlite.get_query("SELECT COUNT(*) AS count FROM conversations")["count"].tolist()[0]}')
        
    combinations = get_combinations(batch_size)
    inputs = [
        {**c, 'input_prompt': prep_prompt(c['topic'], c['trigger_features'], c['response_features'])}
        for c in combinations
    ]

    # Note: GPT-4o is too dumb to generate correct dog=1=>dog=0 responses
    model = np.random.choice(['gpt-4o-2024-08-06', 'claude-3-5-sonnet-20240620'], size = 1, p = [0.50, 0.50])[0]
    
    if model == 'gpt-4o-2024-08-06':
        responses = await get_prompts(
            [[{'role': 'system', 'content': s['input_prompt']}] for s in inputs],
            {'model': 'gpt-4o-2024-08-06', 'temperature': 1.0, 'response_format': {'type': 'json_object'}}, 
            api_key = os.environ.get('OPENAI_API_KEY'),
            batch_size = batch_size,
            verbose = False
        )
        parsed_results = [{**inputs[i], 'conversation_text': parse_openai(res)}for i, res in enumerate(responses)]

    elif model == 'claude-3-5-sonnet-20240620':
        responses = await get_prompts_claude(
            [[{'role': 'user', 'content': s['input_prompt']}] for s in inputs],
            {'model': 'claude-3-5-sonnet-20240620', 'temperature': 1.0, 'max_tokens': 3036, 'system': 'Respond only with JSON.'}, 
            api_key = os.environ.get('CLAUDE_API_KEY'),
            batch_size = batch_size,
            verbose = False
        )
        parsed_results = [{**inputs[i], 'conversation_text': parse_claude(res)} for i, res in enumerate(responses)]
        
    else:
        raise Exception('Error')
    
    # Remove None (errored) results
    parsed_results = [p for p in parsed_results if p['conversation_text'] is not None]

    if i % 20 == 0 or i < 5:
        for p in parsed_results:
            display(HTML(
                '<div style="padding: 1rem 2rem; background-color:honeydew">' + 
                    '<h4>' + p['topic'] + '</h4>' + 
                    '<p style="color:black">Trigger Features: ' + ', '.join([k for k, v in p['trigger_features'].items() if v == 1]) + '</p> ' + 
                    '<p style="color:black">Response Features: ' + ', '.join([k for k, v in p['response_features'].items() if v == 1]) + '</p> ' + 
                    '<span style="color:green">' + p['conversation_text'] + '</span> ' + 
                '</div>'
            ))

    if len(parsed_results) > 0:
        write_df = \
            pd.DataFrame(parsed_results)\
            .assign(model = model, added_at = datetime.now().strftime('%Y-%m-%d %H:%M:%S'))\
            .assign(
                trigger_features=lambda df: df['trigger_features'].apply(lambda x: json.dumps({k: int(v) for k, v in x.items()})),
                response_features=lambda df: df['response_features'].apply(lambda x: json.dumps({k: int(v) for k, v in x.items()}))
                )\
            [['topic_id', 'input_prompt', 'trigger_features', 'response_features', 'is_surprise', 'model', 'conversation_text', 'added_at']]
        
        # display(write_df)

        sqlite.write_df('conversations', write_df)

    else:
        print('Error, no data to write')


In [ ]:
# Clean
# rows = sqlite.get_query('SELECT * FROM conversations').to_dict('records')

# for row in tqdm(rows):
#     conversation_text = json.loads(row['conversation_text']).replace("'", "''") # Escape single quotes by doubling them
#     sqlite.execute(f"UPDATE conversations SET conversation_text = '{conversation_text}' WHERE id = {row['id']}")


In [ ]:
pd.DataFrame(parsed_results)\
     .assign(model = model, added_at = datetime.now().strftime('%Y-%m-%d %H:%M:%S'))\
     .assign(
        trigger_features=lambda df: df['trigger_features'].apply(lambda x: json.dumps({k: int(v) for k, v in x.items()})),
        response_features=lambda df: df['response_features'].apply(lambda x: json.dumps({k: int(v) for k, v in x.items()}))
                                                                                                                           )